In [1]:
import fastai
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.script import *
import fastai.vision.learner as fvl
from fastai.vision.learner import cnn_config
from fastai.vision.models.unet import DynamicUnet

from skimage import filters
from skimage.util import random_noise
from functools import partial
from model.metrics import *

# Choose GPU Device

In [2]:
torch.cuda.set_device(1)

# Set Experiment Name

In [3]:
nb_name = 'test'

# Load Data

In [4]:
from data.load import get_data

### Parameters for Data Loading

In [5]:
bs = 8  # batch size
size = 512  # image size
data_pth = Path('/home/alaa/Dropbox (BPHO)/BPHO Staff/USF')  # path to dataset
lr_dir = f'EM/training/trainsets/crappified/'  # input images
hr_dir = f'EM/training/trainsets/hr/'  # target images
model_path = data_pth/f'EM/models/'  # path to saved models

In [9]:
# Store dataset in a databunch object 
data = get_data(data_pth, lr_dir, hr_dir, bs, 
                in_sz=size, out_sz=size, max_zoom=3)

/home/alaa/Dropbox (BPHO)/BPHO Staff/USF/EM/training/trainsets/crappified


In [6]:
from model.metrics import psnr, ssim
metrics = [F.mse_loss, psnr, ssim]

# Modelling

## Load Model (optional)

In [10]:
arch = models.resnet34
wd = 1e-3
learn = unet_learner(data, arch, wd=wd, 
                     #loss_func=feat_loss,
                     loss_func=F.mse_loss,
                     metrics=metrics, 
                     #callback_fns=LossMetrics, 
                     blur=True, norm_type=NormType.Weight, model_dir=model_path)
gc.collect()

7

In [20]:
expt_num = '3b'
learn = learn.load(f'{nb_name}.{expt_num}')

## Model Training

In [7]:
def do_fit(save_name, lrs=1e-3, pct_start=0.9, cycle_len=10):
    learn.fit_one_cycle(cycle_len, lrs, pct_start=pct_start)
    learn.save(save_name)
    num_rows = min(learn.data.batch_size, 3)
    learn.show_results(rows=num_rows, imgsize=5)

In [ ]:
# Learning rate finder
learn.lr_find()

In [ ]:
# Visualize LR finder results, choose a lr value
learn.recorder.plot()

In [ ]:
lr = 3e-4  # put LR value here
# start training
do_fit(f'{nb_name}.1', lr, cycle_len=1)